# Test


In [34]:
import pandas as pd
import matplotlib.pyplot as plt

# read data:
df = pd.read_excel('Bioassay_profit.xlsx')

# Rename columns:
df.rename(columns={
    'Project Code': 'project',
    'Project Type': 'type',
    'Date Completed\n(e.g. report sent)': 'date',
    'Bioassay Contract Value ($)*': 'contract_value',
    'Profit ($)': 'profit'
}, inplace=True)


# Set types:
df.contract_value = df.contract_value.astype(float)
df.profit = df.profit.astype(float)

print(f'Number of projects all together: {len(df)}')
print('\nThe following rows are having missingness and will be deleted:')
print(df.loc[(df.contract_value.isna()) 
            | (df.profit.isna()) 
            | (df.contract_value == 0)].drop(['date'], axis=1).sort_values(['type']))

# Remove missing values: 
df = df.loc[(df.contract_value.notna()) 
            & (df.profit.notna()) 
            & (df.contract_value > 0)]

print(f'Number of projects after removing missing values: {len(df)}')
print('\nThe following rows have matching contract values and profit values:')
print(df.loc[df.contract_value == df.profit].drop(['date'], axis=1).sort_values(['type']))

# Remove rows where the contract value is equal to the profit:
df = df.loc[df.contract_value != df.profit]

print(f'\n\nNumber of projects left: {len(df)}')

# Convert date to year:
df['year'] = df.date.dt.year

# Adding labels for each project type:
project_types = df.type.unique().tolist()
project_types.sort()
df['type_index'] = df.type.apply(lambda x: project_types.index(x))


# How "expensive is the profit? - tells the ratio of the profit/contract value."
df['cost'] = df.contract_value - df.profit
df['profit_cost'] = df.apply(lambda row: row['profit']/row['cost'] if row['profit'] > 0 else None, axis=1)

df.to_csv('cleaned_profit.tsv', sep='\t', index=False, na_rep='NAN')

Number of projects all together: 167

The following rows are having missingness and will be deleted:
Empty DataFrame
Columns: [project, type, contract_value, profit]
Index: []
Number of projects after removing missing values: 167

The following rows have matching contract values and profit values:
Empty DataFrame
Columns: [project, type, contract_value, profit]
Index: []


Number of projects left: 167


In [14]:

# Drawing project profit:
plt.figure(figsize=(13,6))
ax = plt.scatter(df.type_index, df.profit)
plt.box(False)
plt.axis('off')
plt.show()


In [16]:
%load_ext rpy2.ipython

/Users/dsuveges/opt/anaconda3/envs/jupyter/lib/python3.6/site-packages/rpy2/robjects/pandas2ri.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


In [295]:
%%R 

df = read.table('cleaned_profit.tsv', sep='\t', header=TRUE, 
                comment.char = "%", stringsAsFactors=FALSE, as.is=TRUE)

print(head(df))

  project        type       date contract_value   profit year type_index
1   OPD02   Other FFS 2018-03-22        18200.0  -6177.6 2018          8
2   ASL02   Other FFS 2018-04-19        20956.0 -10751.0 2018          8
3  DIU100   Other FFS 2018-04-20       137280.0  47086.0 2018          8
4   FAR07   Other FFS 2018-04-27        51980.5  12043.2 2018          8
5   FAR14   Other FFS 2018-07-12        20813.0   1990.3 2018          8
6  CRT100 ADC Testing 2018-07-16        15340.0   4270.5 2018          0
     cost         profit_cost
1 24377.6                 NAN
2 31707.0                 NAN
3 90194.0  0.5220524646872298
4 39937.3  0.3015526838319064
5 18822.7 0.10573934663996128
6 11069.5  0.3857897827363476


In [280]:
%%R -w 700 -h 600

get_color_gradient = function(profit_values){
    colfunc_gain = colorRampPalette(c("white","cornflowerblue"))
    colfunc_loss = colorRampPalette(c("white", "firebrick"))

    profit_min = min(profit_values, na.rm=TRUE)
    profit_max = max(profit_values, na.rm=TRUE)
    
    col_values = c()
    counter = 0
    for (profit in profit_values){
        if (is.na(profit)){
            col_values = append(col_values, NA)
        }
        else{
            if(profit >= 0){
                col_values = append(col_values, colfunc_gain(100)[ceiling(100*profit/profit_max)])
            }
            else{
                col_values = append(col_values, colfunc_loss(100)[ceiling(100*profit/profit_min)])                
            }
        }
        counter = counter + 1
    }
    return (col_values)
}
               

plot_project_type_profit = function(d, plot_label){

    # Get list of all project types:
    project_types = unique(df[,c('type','type_index')])
    project_types = project_types[order(project_types$type_index),]

    # Aggregate profit:
    agg = aggregate(profit~type, d, sum)

    # Merge aggregate with all project types:
    agg = merge(agg, project_types, by = "type", all = TRUE)

    # Setting colors
    agg['col'] = ifelse(agg$profit > 0,
        adjustcolor('cornflowerblue', alpha.f = 0.5),
        adjustcolor('firebrick', alpha.f = 0.5))

    # Get parameters:
    b = barplot(agg$profit, plot=FALSE)
    agg['xpos'] = b

    # get x/y pairs:
    points = merge(d[,c("type","profit")], agg[,c("type","xpos")], by = "type", all = TRUE)

    # Generating colors based on 
    # points['col'] = get_color_gradient(points$profit)
    points['col'] = ifelse(points$profit>0,
       adjustcolor('cornflowerblue', alpha.f = 0.5),
        adjustcolor('firebrick', alpha.f = 0.5))

    ##
    ## Setting up plotting area:
    ##
    pdf(sprintf("project_type_profit_%s.pdf", plot_label))
    par(mar=c(1,5.5,3.1,2.1), mfrow = c(2,1))

    plot(points$xpos, points$profit,, xlab='', ylab='', cex.lab=1.3,col='white',
         xlim=c(0,max(points$xpos)),axes=FALSE,
        xaxt='n',las=2, main=plot_label)
    axis(2, las=2)
    mtext('Profit/project', side=2, line=4.2, cex=1.2)

    # Adding horizontal line at 0 profit:
    abline(h=0, col='red')


    points(points$xpos, points$profit, pch=21, cex=1.4, bg=points$col)

    # Adding barchart:
    par(mar=c(7,5.5,0.5,2.1))
    barplot(agg$profit, las =2,xaxs="r", xlim=c(0,max(points$xpos)), col=agg$col)
    mtext('Sum Profit', side=2, line=4.2, cex=1.2)

    mtext(agg$type, side=1, line=1, cex=0.9, at=agg$xpos, las=2)
    dev.off()
    
}

# 
df["profit_cost"] = df$profit / df$cost

# Plotting all data:
plot_project_type_profit(df, 'full')

# Generate plot for all years:
for( year in unique(df$year)){
    plot_project_type_profit(df[df$year == year,],year)
}




In [337]:
%%R -w 700
df = read.table('cleaned_profit.tsv', sep='\t', header=TRUE, 
                comment.char = "%", stringsAsFactors=FALSE, as.is=TRUE)

df$profit_cost = as.numeric(df$profit_cost)
plot(x= as.numeric(rownames(df)),y=df$profit_cost, col=adjustcolor('firebrick', alpha=0.5) , pch=16, cex=2)
# df$profit_cost

In [376]:
%%R -w 700

# year = 2020
# d = df[df$year == year,]
# plot_label = year

plot_profit_cost = function(d, plot_label){
    # Get list of all project types:
    project_types = unique(df[,c('type','type_index')])
    project_types = project_types[order(project_types$type_index),]

    # Aggregate profit:
    agg = aggregate(profit_cost~type, d, median)

    # Merge aggregate with all project types:
    agg = merge(agg, project_types, by = "type", all = TRUE)

    # Setting colors
    agg['col'] = ifelse(agg$profit_cost > 0,
        adjustcolor('cornflowerblue', alpha.f = 0.5),
        adjustcolor('firebrick', alpha.f = 0.5))

    # Get parameters:
    b = barplot(agg$profit_cost, plot=FALSE)
    agg['xpos'] = b

    # get x/y pairs:
    points = merge(d[,c("type","profit_cost", "profit")], agg[,c("type","xpos")], by = "type", all = TRUE)
    points["point_size"] = (abs(points$profit) / max(abs(df$profit), rm.na=TRUE))*5+1

    # Generating colors based on 
    # points['col'] = get_color_gradient(points$profit)
    points['col'] = ifelse(points$profit_cost>0,
       adjustcolor('cornflowerblue', alpha.f = 0.5),
        adjustcolor('firebrick', alpha.f = 0.5))
    points = points[order(points$point_size),]

    ##
    ## Setting up plotting area:
    ##
    pdf(sprintf("project_type_profit_cost_%s.pdf", plot_label))
    par(mar=c(2,5.5,3.1,2.1), mfrow = c(2,1))

    plot(points$xpos, points$profit_cost,, xlab='', ylab='', cex.lab=1.3,col='white',
         xlim=c(0,max(points$xpos)),axes=FALSE,
        xaxt='n',las=2, main=plot_label)
    axis(2, las=2)
    mtext('profit/cost', side=2, line=4.2, cex=1.2)

    # Adding horizontal line at 0 profit:
    abline(h=0, col='red')


    points(points$xpos, points$profit_cost, pch=21, cex=points$point_size, bg=points$col)

    # Adding barchart:
    par(mar=c(7,5.5,0.5,2.1))
    barplot(agg$profit_cost, las =2,xaxs="r", xlim=c(0,max(points$xpos)), col=agg$col)
    mtext('Median profit/cost', side=2, line=4.2, cex=1.2)

    mtext(agg$type, side=1, line=1, cex=0.9, at=agg$xpos, las=2)
    dev.off()
}


# Plotting all data:
plot_profit_cost(df, 'full')

# Generate plot for all years:
for( year in unique(df$year)){
    plot_profit_cost(df[df$year == year,],year)
}

In [378]:
%%R

d

      project            type       date contract_value     profit year
100     FAR11       Other FFS 2020-01-09       52580.00  13803.600 2020
101 ASZA-1276            TCEM 2020-01-10       87500.00  21497.700 2020
102 TRZZ-1306        Fc assay 2020-01-10       18465.00  10456.350 2020
103 NGLS-1279            DC:T 2020-01-14      119000.00  26836.500 2020
104 CRDO-1304 Cytokine Screen 2020-01-16       20390.00   2091.200 2020
105 DIMC-1183       Other FFS 2020-01-16      100000.00  53205.200 2020
106 BOIG-1310     Time Course 2020-01-17       71500.00  64598.300 2020
107     NCS05     Time Course 2020-01-17       59150.00   7619.300 2020
108 NECE-1103 Cytokine Screen 2020-01-17       18600.00   2113.400 2020
109 BOIG-1310           MAPPs 2020-01-24       40200.00  30777.600 2020
110 MANC-1358       In silico 2020-01-29        6500.00   6052.150 2020
111 ADPH-1184     ADC Testing 2020-02-10       46537.00  16091.000 2020
112 EISI-1285     Time Course 2020-02-10      126500.00  45696.2

NameError: name 'd' is not defined